## Load Raw data From Bronze layer to Sliver Layer

In [2]:
# Load raw data into Bronze layer
silver_df_master = spark.read.csv("Files/data/master_dataframe.csv", header=True, inferSchema=True)
silver_df_by_ball = spark.read.csv("Files/data/ipl_ball_by_ball_output_raw.csv", header=True, inferSchema=True)
silver_df_summary = spark.read.csv("Files/data/ipl_summary_raw.csv", header=True, inferSchema=True)

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 4, Finished, Available, Finished)

In [3]:
display(silver_df_summary)

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0adc4845-2ff8-42fd-bf46-d4fa92e6dcd9)

## Streamline venue names by removing city names from the venue

In [5]:
from pyspark.sql.functions import col, regexp_replace

silver_df_summary_cleaned = silver_df_summary.withColumn(
    "info_venue",
    regexp_replace(col("info_venue"), ",.*$", "")
)

# Show the cleaned venue names
silver_df_summary_cleaned.select("info_venue").distinct().show

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 7, Finished, Available, Finished)

<bound method DataFrame.show of DataFrame[info_venue: string]>

In [6]:
display(silver_df_summary_cleaned)

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8b0fd684-a761-4c2c-b97a-507d37dd9ef2)

In [10]:
from pyspark.sql.functions import col, when

# Transformations for City Name, Team Name, and Stadium Name Changes
silver_df_summary_cleaned = silver_df_summary.withColumn(
    "info_city", when(col("info_city") == "Bangalore", "Bengaluru")
               .otherwise(col("info_city"))
).withColumn(
    "info_teams_1", when(col("info_teams_1") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_1") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_1"))
).withColumn(
    "info_teams_2", when(col("info_teams_2") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_2") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_2"))
).withColumn(
    "info_toss_winner", when(col("info_toss_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_toss_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_toss_winner"))      
).withColumn(
    "info_outcome_winner", when(col("info_outcome_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_outcome_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_outcome_winner"))            
).withColumn(
    "info_venue", when(col("info_venue") == "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium")
             .otherwise(col("info_venue"))
).na.fill("Unknown", subset=["info_venue", "info_city", "info_teams_1", "info_teams_2", "info_toss_winner","info_outcome_winner"])

silver_df_summary_cleaned.select("info_venue").distinct().show()

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 12, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|Dubai Internation...|
|Himachal Pradesh ...|
|M Chinnaswamy Sta...|
|Sardar Patel Stad...|
|Punjab Cricket As...|
|Punjab Cricket As...|
|    Barabati Stadium|
|Punjab Cricket As...|
|       Nehru Stadium|
|Maharashtra Crick...|
|Brabourne Stadium...|
|        Eden Gardens|
|Arun Jaitley Stad...|
|     OUTsurance Oval|
|M Chinnaswamy Sta...|
|Punjab Cricket As...|
|    Feroz Shah Kotla|
|Sawai Mansingh St...|
|Rajiv Gandhi Inte...|
|Bharat Ratna Shri...|
+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import regexp_replace, trim

# Clean venue names by removing city or extra descriptions after commas
# We want to keep only the main venue name, even if there are commas or additional details
silver_df_summary_cleaned = silver_df_summary.withColumn(
    "info_venue",
    trim(
        regexp_replace(col("info_venue"), r",.*$", "")
    )
)

# Manually handle cases where venues might still need extra corrections
# For example, handle specific cases like "MA Chidambaram Stadium" without the city part
venue_cleaning_rules = [
    (r"Rajiv Gandhi International Stadium.*", "Rajiv Gandhi International Stadium"),
    (r"Punjab Cricket Association IS Bindra Stadium.*", "Punjab Cricket Association IS Bindra Stadium"),
    (r"MA Chidambaram Stadium.*", "MA Chidambaram Stadium"),
    # Add any other complex venues here
]

# Apply the specific cleaning rules
for pattern, replacement in venue_cleaning_rules:
    silver_df_summary_cleaned = silver_df_summary_cleaned.withColumn(
        "info_venue",
        regexp_replace(col("info_venue"), pattern, replacement)
    )

# Show the cleaned venue names to verify
silver_df_summary_cleaned.select("info_venue").distinct().show()


StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 10, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|Dubai Internation...|
|Himachal Pradesh ...|
|Punjab Cricket As...|
|Vidarbha Cricket ...|
|    Barabati Stadium|
|       Nehru Stadium|
|Maharashtra Crick...|
|        Eden Gardens|
|     OUTsurance Oval|
|M Chinnaswamy Sta...|
|    Feroz Shah Kotla|
|   Brabourne Stadium|
|Barsapara Cricket...|
|M.Chinnaswamy Sta...|
|          Green Park|
|Holkar Cricket St...|
|Bharat Ratna Shri...|
|Shaheed Veer Nara...|
|Sheikh Zayed Stadium|
|Sharjah Cricket S...|
+--------------------+
only showing top 20 rows



In [11]:
display(silver_df_summary_cleaned)

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8abec653-19c1-4614-9691-df2eaa134f92)

In [12]:
from delta.tables import DeltaTable

# Create or update Silver table if not exists
silver_table = DeltaTable.createIfNotExists(spark).location("Tables/silver_summary_table").execute()

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 14, Finished, Available, Finished)

In [13]:
# Write the transformed data into the Silver layer
silver_df_summary_cleaned.write.option("overwriteSchema", "true").format("delta").mode("overwrite").save("Tables/silver_summary_table")

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 15, Finished, Available, Finished)

In [14]:
from pyspark.sql.functions import regexp_replace, trim, col

# Clean the venue names in bronze_df_master
sliver_df_master_cleaned = silver_df_master.withColumn(
    "venue_cleaned",
    trim(
        regexp_replace(col("venue"), r"([a-z])([A-Z])", r"$1 $2")
    )
)

# Show cleaned venue names
sliver_df_master_cleaned.select("venue", "venue_cleaned").distinct().show()

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 16, Finished, Available, Finished)

+--------------------+--------------------+
|               venue|       venue_cleaned|
+--------------------+--------------------+
|                null|                null|
|      SuperSportPark|    Super Sport Park|
| NewWanderersStadium|New Wanderers Sta...|
|      StGeorge'sPark|    St George's Park|
|         EdenGardens|        Eden Gardens|
|    BrabourneStadium|   Brabourne Stadium|
|            Newlands|            Newlands|
|SawaiMansinghStadium|Sawai Mansingh St...|
|DrDYPatilSportsAc...|Dr DYPatil Sports...|
|     BarabatiStadium|    Barabati Stadium|
|  SardarPatelStadium|Sardar Patel Stadium|
|  DeBeersDiamondOval|De Beers Diamond ...|
|      OUTsuranceOval|     OUTsurance Oval|
|MAChidambaramStadium|MAChidambaram Sta...|
|     WankhedeStadium|    Wankhede Stadium|
| MChinnaswamyStadium|MChinnaswamy Stadium|
|         BuffaloPark|        Buffalo Park|
|      FerozShahKotla|    Feroz Shah Kotla|
|RajivGandhiIntern...|Rajiv Gandhi Inte...|
|PunjabCricketAsso...|Punjab Cri

In [15]:
from pyspark.sql.functions import col, when

# Transformations for City Name, Team Name, and Stadium Name Changes
sliver_df_master_cleaned = silver_df_master.withColumn(
    "city", when(col("city") == "Bangalore", "Bengaluru")
               .otherwise(col("city"))
).withColumn(
    "team_1", when(col("team_1") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("team_1") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("team_1"))
).withColumn(
    "team_2", when(col("team_2") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("team_2") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("team_2"))
).withColumn(
    "toss_winner", when(col("toss_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("toss_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("toss_winner"))      
).withColumn(
    "match_winner", when(col("match_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("match_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("match_winner"))            
).withColumn(
    "venue", when(col("venue") == "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium")
    .when(col("venue") == "MChinnaswamy Stadium", "M Chinnaswamy Stadium")
    .when(col("venue") == "MAChidambaram Stadium", "MA Chidambaram Stadium")
             .otherwise(col("venue"))
).na.fill("Unknown", subset=["venue", "city", "team_1", "team_2", "toss_winner","match_winner"])

sliver_df_master_cleaned.select("venue","city", "team_1", "team_2", "toss_winner","match_winner").distinct().show()

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 17, Finished, Available, Finished)

+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|               venue|          city|              team_1|              team_2|         toss_winner|        match_winner|
+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|      StGeorge'sPark|Port Elizabeth|Royal Challengers...|      Delhi Capitals|Royal Challengers...|      Delhi Capitals|
|      SuperSportPark|     Centurion| Chennai Super Kings|    Rajasthan Royals|    Rajasthan Royals| Chennai Super Kings|
| MChinnaswamyStadium|     Bengaluru|      Delhi Capitals|Royal Challengers...|Royal Challengers...|      Delhi Capitals|
|SawaiMansinghStadium|        Jaipur|    Rajasthan Royals|Kolkata Knight Ri...|    Rajasthan Royals|    Rajasthan Royals|
| NewWanderersStadium|  Johannesburg|     Deccan Chargers|Royal Challengers...|Royal Challengers...|     Deccan Chargers|
| MChinnaswamyStadium|  

In [16]:
from delta.tables import DeltaTable

# Create or update Silver table if not exists
silver_master_table = DeltaTable.createIfNotExists(spark).location("Tables/silver_master_table").execute()

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 18, Finished, Available, Finished)

In [17]:
# Write the transformed data into the Silver layer
sliver_df_master_cleaned.write.option("overwriteSchema", "true").format("delta").mode("overwrite").save("Tables/silver_master_table")

StatementMeta(, 85c2bf9e-4ae2-4cc7-bb87-9d4b74644eb6, 19, Finished, Available, Finished)